In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os

##  Integral Image

Computes rectangle integrals of an image for fast computation of Harr Filters.

In [2]:
def compute_intg_image(img):
    """
    Computes matrix for quick computation of integrals.
    """
    # Create a matrix of zeros with the same dimensions as the input image
    intg_img = np.zeros(img.shape, dtype=np.int64)
    
    # Iterate over each pixel in the input image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the sum of all pixels above and to the left of the current pixel
            if i > 0:
                intg_img[i, j] += intg_img[i-1, j]
            if j > 0:
                intg_img[i, j] += intg_img[i, j-1]
            if i > 0 and j > 0:
                intg_img[i, j] -= intg_img[i-1, j-1]
            
            # Add the value of the current pixel to the integral image
            intg_img[i, j] += img[i, j]
    
    # Return the computed integral image
    return intg_img

def get_rect(intg_img, si, sj, ei, ej):
    # Calculate the sum of all pixels within the specified rectangular region
    result = intg_img[ei, ej]
    if si > 0:
        result -= intg_img[si-1, ej]
    if sj > 0:
        result -= intg_img[ei, sj-1]
    if si > 0 and sj > 0:
        result += intg_img[si-1, sj-1]
    
    # Return the computed sum of pixels
    return result

In [3]:
# (height, width), stride (y, x) [rect: pos start (i,j), rect pos end (i,j), wt]
harr_filter_1 = [(20,40), (5, 5) ,[(0, 0, 9, 39, -1) , (10,0,19,39, 1)]]
harr_filter_2 = [(40,10), (5, 5) ,[(0, 0, 39, 4, 1) , (0,5,39,9, -1)]]

harr_filter_3 = [(24,40), (5, 5) ,[(0, 0, 7, 39, -0.5), (8, 0, 15, 39, 1), (16, 0, 23, 39, -0.5)]]

harr_filter_4 = [(40,24), (5, 5) ,[(0, 0, 39, 7, -0.5), (0, 8, 39, 15, 1), (0, 16, 39, 23, -0.5)]]



def run_filter(c_img , intg_img, filter):
    result_img = np.zeros((intg_img.shape[0]//filter[1][0], intg_img.shape[1]//filter[1][1])) 
    
    for i in range(0, intg_img.shape[0]-filter[0][0]+1, filter[1][0]):
        for j in range(0, intg_img.shape[1]-filter[0][1]+1, filter[1][1]):
            # r_img = c_img.copy()
            if (i+filter[0][0]) < intg_img.shape[0] and (j + filter[0][1]) < intg_img.shape[1]:
                result = 0
                for rect_filter in filter[2]:
                    # cv2.rectangle(r_img, (j+rect_filter[1],i+rect_filter[0]), (j+rect_filter[3],i+rect_filter[2]), (255,0,0), 1)
                    result += (rect_filter[4] * get_rect(intg_img, i+rect_filter[0], j+rect_filter[1], i+rect_filter[2], j+rect_filter[3]) ) / ((rect_filter[2]-rect_filter[0]+1)*(rect_filter[3]-rect_filter[1]+1))
                result_img[i//filter[1][0],  j//filter[1][1]] = result
            
       
            
     
    return result_img
fal_1 = run_filter(c_img, intg_img, harr_filter_4)
print(fal_1.max())
fal_1 = cv2.threshold(fal_1, 10, 255,cv2.THRESH_BINARY)[1]


cv2.imshow("set", fal_1)
cv2.imshow("orig",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'c_img' is not defined

In [4]:
def create_haar_filter(height, width, white_subrects, black_subrects, stride=(5,5)):
    # Initialize:
    # 1. An empty list to store the sub-rectangles
    rects = []
    # 2. A variable to count the number of white sub-rectangles
    white_count = 0
    # 3. A variable to count the number of black sub-rectangles                 
    black_count = 0
    
    # Loop over the white sub-rectangles and compute the total number of white pixels
    for subrect in white_subrects:
        white_count += (subrect[2]-subrect[0]+1)*(subrect[3]-subrect[1]+1)
    
    # Loop over the black sub-rectangles and compute the total number of black pixels
    for subrect in black_subrects:
        black_count += (subrect[2]-subrect[0]+1)*(subrect[3]-subrect[1]+1)
    
    # Loop over the white sub-rectangles and append their coordinates and weights to the list of sub-rectangles
    for subrect in white_subrects:
        rects.append([subrect[0], subrect[1], subrect[2], subrect[3], -1/white_count])
    
    # Loop over the black sub-rectangles and append their coordinates and weights to the list of sub-rectangles
    for subrect in black_subrects:
        rects.append([subrect[0], subrect[1], subrect[2], subrect[3], 1/black_count])
    
    # Return a list of the filter kernel dimensions, stride, and sub-rectangles with their weights
    return [(height, width), stride, rects]


In [5]:
def compute_haar_features(intg_img, haar_filter):
    # Get the shape of the integral image and the Haar filter
    img_shape = intg_img.shape
    filter_shape = haar_filter[0]

    # Get the stride of the Haar filter
    stride = haar_filter[1]

    # Create an empty result image with the appropriate shape
    res_img = np.zeros((img_shape[0]//stride[0], img_shape[1]//stride[1]))

    # Iterate over the image with the Haar filter
    for i in range(0, img_shape[0]-filter_shape[0]+1, stride[0]):
        for j in range(0, img_shape[1]-filter_shape[1]+1, stride[1]):
            # If the Haar filter fits inside the image at this position
            if (i+filter_shape[0]) < img_shape[0] and (j + filter_shape[1]) < img_shape[1]:
                result = 0
                # Compute the Haar feature by summing over the rectangles in the Haar filter
                for rect in haar_filter[2]:
                    result += rect[4] * get_rect(intg_img, i+rect[0], j+rect[1], i+rect[2], j+rect[3])
                # Set the result of the Haar feature in the appropriate position in the result image
                res_img[i//stride[0],  j//stride[1]] = result

    # Return the result image
    return res_img

def render_filter(haar_filter):
    # Create an empty image with the shape of the Haar filter
    img = np.zeros(haar_filter[0])

    # Iterate over the rectangles in the Haar filter
    for rect in haar_filter[2]:
        # If the rectangle has a negative weight, mark it as white in the image
        if(rect[4] < 0):
            img[rect[0]:rect[2]+1, rect[1]:rect[3]+1] = 1

    # Display the image with a grayscale colormap
    plt.imshow(img, cmap='gray')
    plt.show()

In [6]:
harr_filter_1 = [(20,40), (5, 5) ,[(0, 0, 9, 39, -1) , (10,0,19,39, 1)]]
harr_filter_2 = [(40,10), (5, 5) ,[(0, 0, 39, 4, 1) , (0,5,39,9, -1)]]

harr_filter_3 = [(24,40), (5, 5) ,[(0, 0, 7, 39, -0.5), (8, 0, 15, 39, 1), (16, 0, 23, 39, -0.5)]]

harr_filter_4 = [(40,24), (5, 5) ,[(0, 0, 39, 7, -0.5), (0, 8, 39, 15, 1), (0, 16, 39, 23, -0.5)]]

# class 1
def haar_c1_h_filter(height, width):
    return create_haar_filter(height, width, [ (0,0, height//2-1, width-1) ], [ (height//2, 0, height-1, width-1) ])
def haar_c1_v_filter(height, width):
    return create_haar_filter(height, width, [ (0,0, height-1, width//2-1) ], [ (0, width//2, height-1, width-1) ])

# class 2
def haar_c2_h_filter(height, width):
    return create_haar_filter(height, width, [ (0, 0, height//3-1, width-1),  (2*height//3, 0, height-1, width-1)  ], [ (height//3, 0, 2*height//3-1, width-1) ])
def haar_c2_v_filter(height, width):
    return create_haar_filter(height, width, [ (0, 0, height-1, width//3-1),  (0, 2*width//3, height-1, width-1)  ], [ (0, width//3, height-1, 2*width//3-1) ])
def haar_c2_s_filter(height, width):
    return create_haar_filter(height, width, [ (0,0, width//2-1, height//2-1), (height//2, width//2, height-1, width-1)], [(0, width//2, height//2-1, width-1), (height//2, 0, width//2-1, height-1)])

# class 3

def haar_c3_s_filter(height, width, factor):
    return create_haar_filter(height, width, [ (0,0, (width-width*factor)//2, height//2-1), (height//2, width//2, height-1, width-1)], [(0, width//2, height//2-1, width-1), (height//2, 0, width//2-1, height-1)])


In [63]:
import cv2
  
  
# define a video capture object
vid = cv2.VideoCapture(0)
haar_filters = [haar_c1_h_filter(20,50), haar_c1_v_filter(60,30), haar_c2_h_filter(21,40), haar_c2_v_filter(60,30), haar_c2_s_filter(30,30)]
thresholds = [20,40,10,10]
colours = [np.random.random(3) for i in range(4)]
k_rows = 1
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (200,200))
    intg_img = compute_intg_image(img)
    draw_grid(img)
    #filt_imgs = [cv2.threshold(compute_haar_features(intg_img, filt), thresh, 255, cv2.THRESH_BINARY)[1] for thresh, filt in zip(thresholds, haar_filters) ]
    filt_imgs = [compute_haar_features(intg_img, filt) for 
                 filt in  haar_filters ]
    
    filt_imgs = [ cv2.threshold(img, 0, 0, cv2.THRESH_TOZERO)[1] / np.max(img) for img in filt_imgs]
    filt_img = np.vstack([np.hstack([filt_imgs[i+j*k_rows] for i in range(k_rows)]) for j in range(len(haar_filters)//k_rows)])
    
    # Display the resulting frame
    cv2.imshow('frame', img)
    cv2.imshow("HARR FILTERS",cv2.resize(filt_img, (200*k_rows, 200*len(haar_filters)//k_rows)))
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    cv2.waitKey(2)
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import sklearn


positive =0
negative =0
wt=np.zeros(img_list.shape)
for i in range(img_list.shape):
    if img_list[i]==1:
        positive +=1
    else:
        negative +=1
for i in range(img_list.shape):
    if img_list[i]==1:
        wt[i]=1/(2*positive)
    else:
        wt[i]=1/(2*negative)



model=sklearn.linear_model.Perceptron(penalty='l2',alpha=0.01)
max_iter=1000

arg_min=np.zeros(max_iter,'uint16')
beta=np.zeros(max_iter,'float')
e=np.zeros(haar.shape[0])
pred=np.zeros((haar.shape[0],haar.shape[2]),'float')

model_list=[0]*haar.shape[0]
best_list=[0]*max_iter

for iter in range(max_iter):
    norm_wt=wt.copy()
    for i in range(img_list.shape):
        norm_wt[i]=wt[i]/np.sum(wt)
    wt=norm_wt
    for j in range(haar.shape[0]):
        # for i in range(img_num):
        model.fit_(harr[j,:,:],img_list)
        pred[j,:]=model.predict(harr[j,:,:])

        model_list[j]=model.copy()

        e[j]=np.dot(wt,np.abs(np.dot(pred[j,:],img_list)))

    arg_min[iter]=np.argmin(e)
    best_list[iter]=model_list[arg_min[iter]].copy()
    beta[iter]=e[arg_min]/(1-e[arg_min])

    new_wt=wt.copy()
    for i in range(haar.shape[2]):
        if(np.round(pred[arg_min[iter],i],2)==img_list):
            new_wt[i]=wt[i]*(beta[iter])
    wt=new_wt


def fun( features ):
    sum_wt=0
    sum_out=0
    for iter in range(max_iter):
        model=best_list[max_iter].copy()
        sum_out+=np.log(beta[iter])*(-model.predict(features))
        sum_wt+=(-0.5)*np.log(beta[iter])
    if(sum_out>=sum_wt):
        return 1
    else:
        return 0
    

    

In [ ]:
img = np.ones((3,3))
intg_img = compute_intg_image(img)

In [7]:
list(range(0,3,1))

[0, 1, 2]

In [67]:
get_rect(intg_img, 0,0, 2,1)

6.0

In [41]:
face_imgs = []
for file in os.listdir('data/'):
    if ".pgm" in file:
        face_imgs.append(cv2.imread("data/"+file,-1))
cv2.imshow("face", np.hstack(face_imgs))
cv2.waitKey(0)

113

In [94]:
def load_folder(folder):
    imgs = []
    for file in os.listdir(folder):
        if ".pgm" in file:
            imgs.append(cv2.imread(folder+"/"+file,-1))
    return imgs

In [77]:
face_imgs[0].shape

(120, 128)

In [27]:
from skimage.feature import haar_like_feature, draw_haar_like_feature, haar_like_feature_coord

In [16]:
haar_like_feature(compute_intg_image(face_imgs[0]),0,0,face_imgs[0].shape[1], face_imgs[0].shape[0])

In [29]:
draw_haar_like_feature(intg_img,0,0,19, 19, haar_like_feature_coord())

TypeError: draw_haar_like_feature() missing 1 required positional argument: 'feature_coord'

In [76]:
intg_img = compute_intg_image(face_imgs[0])

In [77]:
intg_img.shape

(19, 19)

In [40]:
img = face_imgs[0]
kernel_size = 40
for i in range(0, img.shape[0], 10):
    for j in range(0, img.shape[1], 10):
        if i+kernel_size <= img.shape[0] and j+kernel_size <= img.shape[1]:
            cv2.imshow("img",cv2.rectangle(img.copy(), (j, i), (j+kernel_size-1, i+kernel_size-1), 255, 2 ))
            cv2.imshow("roi", cv2.resize(img[i:i+kernel_size, j:j+kernel_size], (100,100)))
            cv2.waitKey(100)

In [105]:
def compute_haar_features(intg_img, haar_filters, feature_size):
    haar_features = np.zeros(feature_size)
    idx = 0
    for filter in haar_filters:
        for i in range(intg_img.shape[0]-filter[0]+1):
            for j in range(intg_img.shape[1]-filter[1]+1):
             
                if filter[2]:
                    result = ( get_rect(intg_img, i, j, i+filter[0]-1, j+filter[1]//2-1) 
                    - get_rect(intg_img, i, j+filter[1]//2, i+filter[0]-1, j+filter[1]-1) )/((filter[0])*filter[1])
                else:
                    result = ( get_rect(intg_img, i, j, i+filter[0]//2-1, j +filter[1]-1) 
                    - get_rect(intg_img, i+filter[0]//2, j, i+filter[0]-1, j+filter[1]-1) )/((filter[0])*filter[1])
                haar_features[idx] = result
                idx += 1
            
        
    return haar_features
def generate_haar_filters(detector_size=19):
    haar_feature_size = 0
    # (r, c, is_vert)
    haar_filters= []
    for i in range(1, detector_size+1):
        for j in range(1, detector_size+1):
            if i % 2 == 0:
                haar_filters.append((i,j, 0))
                haar_feature_size += (19-i+1) * (19-j+1)
            if j % 2 == 0:
                haar_filters.append((i,j, 1))
                haar_feature_size += (19-i+1) * (19-j+1)
    return haar_feature_size, haar_filters

def get_haar_feature_extractor(detector_size):
    haar_feature_size, haar_filters = generate_haar_filters(detector_size)
    def get_haar(intg_img):
        return compute_haar_features(intg_img, haar_filters, haar_feature_size)
    return get_haar

haar_feature_extractor = get_haar_feature_extractor(19)


array([ -6.5       ,  -8.5       , -11.        , ...,   0.96381579,
        -5.00292398,  -0.47076023])

In [90]:
compute_haar_features(intg_img, haar_filters).shape

(34200,)

In [107]:
train_x_true = load_folder("train/face")
train_x_false = load_folder("train/non-face")
train_y_true = [1] * len(train_x_true)
train_y_false = [0] * len(train_x_false)

train_x = train_x_true + train_x_false
train_y = train_y_true + train_y_false

train_x = [compute_intg_image(img) for img in train_x]

6977

In [102]:
6977 * 34200

238613400